<a href="https://colab.research.google.com/github/MadmanMarble/MadmanMarble/blob/main/buildVRT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from osgeo import gdal
import glob

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import shutil

build using chatgpt

In [6]:
# listf('/content/drive/MyDrive/DeepLearning_Project/Datasets/planet_2018_stack/',pattern = '*.tif')
import os
dir_path = '/content/drive/MyDrive/DeepLearning_Project/Datasets/planet_2018_stack/'
[dir_path+f for f in os.listdir(dir_path) if f.endswith('.tif')]

['/content/drive/MyDrive/DeepLearning_Project/Datasets/planet_2018_stack/2018_anahola_stack.tif',
 '/content/drive/MyDrive/DeepLearning_Project/Datasets/planet_2018_stack/2018_haleiwa_stack.tif',
 '/content/drive/MyDrive/DeepLearning_Project/Datasets/planet_2018_stack/2018_hanalei_stack.tif',
 '/content/drive/MyDrive/DeepLearning_Project/Datasets/planet_2018_stack/2018_huleia_stack.tif',
 '/content/drive/MyDrive/DeepLearning_Project/Datasets/planet_2018_stack/2018_kahuku_stack.tif',
 '/content/drive/MyDrive/DeepLearning_Project/Datasets/planet_2018_stack/2018_kakahaia_stack.tif',
 '/content/drive/MyDrive/DeepLearning_Project/Datasets/planet_2018_stack/2018_kaluaapuhi_stack.tif',
 '/content/drive/MyDrive/DeepLearning_Project/Datasets/planet_2018_stack/2018_kamalo_stack.tif',
 '/content/drive/MyDrive/DeepLearning_Project/Datasets/planet_2018_stack/2018_kanaha_stack.tif',
 '/content/drive/MyDrive/DeepLearning_Project/Datasets/planet_2018_stack/2018_kawainui_stack.tif',
 '/content/drive/My

In [17]:
# List of your multi-band TIFF files


dir_path = '/content/drive/MyDrive/DeepLearning_Project/Datasets/planet/2020_select_hawaii/'
[dir_path+f for f in os.listdir(dir_path) if f.endswith('.tif')]


tiff_files = [
    dir_path+f for f in os.listdir(dir_path) if f.endswith('.tif')
]

# Path to the output VRT file
output_vrt = '/content/drive/MyDrive/DeepLearning_Project/Datasets/planet/2020_select_hawaii/2020_select_hawaii_vrt.vrt'

# Create VRT
vrt_options = gdal.BuildVRTOptions(separate=False)  # Use separate=True for multi-band handling
vrt = gdal.BuildVRT(output_vrt, tiff_files, options=vrt_options)
vrt = None  # Close the VRT to flush to disk

end

In [ ]:
# !unzip /content/drive/MyDrive/DeepLearning_Project/Datasets/naip_2021/HI_2021_NAIP_4islands.zip -d /content/drive/MyDrive/DeepLearning_Project/Datasets/naip_2021/

In [ ]:
# Directory containing your .tif files
tif_directory = "/content/drive/MyDrive/DeepLearning_Project/Datasets/planet_hawaii_2020_stack"

# List all .tif files
tif_files = [os.path.join(tif_directory, f) for f in os.listdir(tif_directory) if f.endswith('.tif')]

# Function to get CRS and resolution
def get_crs_and_resolution(file_path):
    dataset = gdal.Open(file_path)
    crs = dataset.GetProjection()
    geotransform = dataset.GetGeoTransform()
    resolution = (geotransform[1], geotransform[5])  # Pixel width and height
    return crs, resolution

# Check CRS and resolution for each file
for file in tif_files:
    crs, resolution = get_crs_and_resolution(file)
    print(f"{file}: CRS={crs}, Resolution={resolution}")


In [ ]:
pip install GDAL


In [ ]:
# Specify the directory containing your .tif files
tif_directory = "/content/drive/MyDrive/DeepLearning_Project/Datasets/planet_2018_stack"

# Specify the output VRT file path
output_vrt_path = os.path.join(tif_directory, "planet_2018_stack_index.vrt")

# List all .tif files in the directory
tif_files = [os.path.join(tif_directory, f) for f in os.listdir(tif_directory) if f.endswith('.tif')]

print(f"Number of files detected: {len(tif_files)}")
for file in tif_files:
    print(file)

# Use GDAL's buildVRT function to create a VRT from the list of tif files
vrt = gdal.BuildVRT(output_vrt_path, tif_files)

# Save and close the VRT
vrt.FlushCache()
vrt = None

print(f"VRT created successfully: {output_vrt_path}")

In [ ]:
# try merging?



end

In [ ]:
#Create VRT, write to disk
path_vrt = gdal.BuildVRT('/content/drive/MyDrive/DeepLearning_Project/Datasets/naip_2021/content/HI_2021_NAIP/naip_index.vrt',
                         glob.glob('/content/drive/MyDrive/DeepLearning_Project/Datasets/naip_2021/content/HI_2021_NAIP/*.tif'))#,options =gdal.BuildVRTOptions(bandList= [1,2,3]))
path_vrt.FlushCache()

# output_raster = gdal.Translate('/content/mosaic.tif',
#                                 '/content/output/gtiff_index.vrt',
#                                 outputSRS='EPSG:3857')
# output_raster.FlushCache()

In [ ]:
path_vrt = gdal.BuildVRT('/content/drive/MyDrive/DeepLearning_Project/Datasets/planet_2018_stack/index_planet_2018.vrt',
                         glob.glob('/content/drive/MyDrive/DeepLearning_Project/Datasets/planet_2018_stack/*.tif'))#,options =gdal.BuildVRTOptions(bandList= [1,2,3]))
path_vrt.FlushCache()

In [ ]:
source = '/content/drive/MyDrive/DeepLearning_Project/Datasets/planet'
destination = '/content/drive/MyDrive/DeepLearning_Project/Datasets/planet_2018'

# Ensure the destination directory exists
if not os.path.exists(destination):
    os.makedirs(destination)

# Loop through all files in the source directory
for filename in os.listdir(source):
    if filename.('.tif'):


